In [ ]:
import pandas as pd
import numpy as np
import math
import pickle

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

import matlab.engine as engi
import matlab as mat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from src import SMOTE
from src import CFS
from src import metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

import matplotlib.pyplot as plt

In [ ]:
def load_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    commit_guru_path = 'data/commit_guru/' + project + '.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
#     print(cols_list)
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    commit_guru_df = pd.read_csv(commit_guru_path)
    cols = understand_df.columns.tolist()
    
    commit_guru_df = commit_guru_df.drop(labels = ['parent_hashes','author_name','author_name',
                                                   'author_email','fileschanged','author_date',
                                                   'author_date_unix_timestamp', 'commit_message',
                                                  'classification', 'fix', 'contains_bug','fixes',],axis=1)

#     print(commit_guru_df.columns)
    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    df = understand_df.merge(commit_guru_df,on='commit_hash')
    cols = df.columns.tolist()
    cols = cols[1:] + [cols[0]]
    df = df[cols]
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.to_csv('data/converted/'+ project + '_understand.csv',index=False)
#     df,cols = apply_cfs(df)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    return X,y
def apply_smote(df):
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

In [ ]:
def norm(x,df):
    lo = df.min()
    hi = df.max()
    return (x - lo) / (hi - lo +0.00000001)

def dominate(_df,t,row_project_name,goals):
    wins = 0
    for i in range(_df.shape[0]):
        project_name = _df.index[i]
        row = _df.iloc[i].tolist()
        if project_name != row_project_name:
            if dominationCompare(row, t,goals,_df):
                wins += 1
    return wins

def dominationCompare(other_row, t,goals,df):
    n = len(goals)
    weight = {'recall':1,'precision':1,'pf':-2.5}
    sum1, sum2 = 0,0
    for i in range(len(goals)):
        _df = df[goals[i]]
        w = weight[goals[i]]
        x = t[i]
        y = other_row[i]
        x = norm(x,_df)
        y = norm(y,_df)
        sum1 = sum1 - math.e**(w * (x-y)/n)
        sum2 = sum2 - math.e**(w * (y-x)/n)
    return sum1/n < sum2/n

In [ ]:
def run_self(project):
    X,y = load_data(project)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = LogisticRegression()
    clf.fit(X_train,y_train)
    predicted = clf.predict(X_test)
    abcd = metrices.measures(y_test,predicted)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    auc = roc_auc_score(y_test, predicted)
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc

In [ ]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

In [ ]:
precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
for project in projects:
    try:
        if project == '.DS_Store':
            continue
    #     if project != 'guice':
    #         continue
        print("+++++++++++++++++   "  + project + "  +++++++++++++++++")
        recall,precision,pf,f1,g_score,auc = run_self(project)
        recall_list[project] = recall
        precision_list[project] = precision
        pf_list[project] = pf
        f1_list[project] = f1
        g_list[project] = g_score
        auc_list[project] = auc
    except Exception as e:
        print(e)
        continue
final_result = {}
final_result['precision'] = precision_list
final_result['recall'] = recall_list
final_result['pf'] = pf_list
final_result['f1'] = f1_list
final_result['g'] = g_list
final_result['auc'] = auc_list
with open('data/self_100.pkl', 'wb') as handle:
    pickle.dump(final_result, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
_precision = list(precision_list.values())
_recall = list(recall_list.values())
_pf = list(pf_list.values())
_f1 = list(f1_list.values())
_g = list(g_list.values())
_auc = list(auc_list.values())
print(np.median(_precision),np.median(_recall),np.median(_pf),np.median(_f1),np.median(_g),np.median(_auc))
fig = plt.figure(num=None, figsize = (20,4), facecolor='w', edgecolor='k')
ax = fig.add_subplot(161)
ax.boxplot(_precision)
ax.set_title('Precision',size = 15)
ax = fig.add_subplot(162)
ax.boxplot(_recall)
ax.set_title('Recall',size = 15)
ax = fig.add_subplot(163)
ax.boxplot(_pf)
ax.set_title('pf',size = 15)
ax = fig.add_subplot(164)
ax.boxplot(_f1)
ax.set_title('f1',size = 15)
ax = fig.add_subplot(165)
ax.boxplot(_g)
ax.set_title('g',size = 15)
ax = fig.add_subplot(166)
ax.boxplot(_auc)
ax.set_title('auc',size = 15)
# fig.savefig('without_process.png')

In [ ]:
def run_self(project):
    X,y = load_data(project)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = LogisticRegression()
    clf.fit(X_train,y_train)
    return clf

In [ ]:
precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
for s_project in projects:
    try:
        if project == '.DS_Store':
            continue
    #     if project != 'guice':
    #         continue
        print("+++++++++++++++++   "  + s_project + "  +++++++++++++++++")
        clf = run_self(s_project)
        if s_project not in precision_list.keys():
            precision_list[s_project] = {}
            recall_list[s_project] = {}
            pf_list[s_project] = {}
            f1_list[s_project] = {}
            g_list[s_project] = {}
            auc_list[s_project] = {}    
        for d_project in projects:
            try:
                X,y = load_data(d_project)
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
                predicted = clf.predict(X_test)
                abcd = metrices.measures(y_test,predicted)
                pf = abcd.get_pf()
                recall = abcd.calculate_recall()
                precision = abcd.calculate_precision()
                f1 = abcd.calculate_f1_score()
                g_score = abcd.get_g_score()
                auc = roc_auc_score(y_test, predicted)
                precision_list[s_project][d_project] = precision
                recall_list[s_project][d_project] = recall
                pf_list[s_project][d_project] = pf
                f1_list[s_project][d_project] = f1
                g_list[s_project][d_project] = g_score
                auc_list[s_project][d_project] = auc
            except Exception as e:
                print('d_project',e)
                continue
    except Exception as e:
        print('s_project',e)
        continue
final_result = {}
final_result['precision'] = precision_list
final_result['recall'] = recall_list
final_result['pf'] = pf_list
final_result['f1'] = f1_list
final_result['g'] = g_list
final_result['auc'] = auc_list
with open('data/conv_bellwether_100.pkl', 'wb') as handle:
    pickle.dump(final_result, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
df_self = pd.read_pickle('data/self_100.pkl')
df_bellwether = pd.read_pickle('data/conv_bellwether_100.pkl')
metrices = df_self.keys()
median = 0
median_results = {}
for s_project in projects:
    try:
        if s_project not in median_results.keys():
            median_results[s_project] = {}
        for metric in metrices:
            median_results[s_project][metric] = np.median(list(df_bellwether[metric][s_project].values()))
    except Exception as e:
        print(s_project)
        continue

In [ ]:
df = pd.DataFrame.from_dict(median_results,orient='index')

In [ ]:
df.to_csv('result/cdom_score_100.csv',index=False)

In [ ]:
dom_score = []
goals = ['recall','precision','pf']
for row_id in range(df.shape[0]):
    project_name = df.index[row_id]
    row = df.iloc[row_id].tolist()
    wins = dominate(df,row,project_name,goals)
    dom_score.append(wins)
df['wins'] = dom_score

In [ ]:
df[df['wins'] == df.wins.max()]

In [ ]:
df_self = pd.read_pickle('data/self_100.pkl')
df_bellwether = pd.read_pickle('data/conv_bellwether_100.pkl')
metrices = df_self.keys()
median = 0
bell_results = {}
for s_project in projects:
    try:
        for metric in metrices:
            if metric not in bell_results.keys():
                bell_results[metric] = {}
            if s_project not in bell_results[metric]:
                bell_results[metric][s_project] = {}
            _self = df_self[metric][s_project]
            _bell = df_bellwether[metric]['disruptor'][s_project]
            bell_results[metric][s_project]['self'] = _self
            bell_results[metric][s_project]['bell'] = _bell
    except Exception as e:
        print(s_project)
        continue

In [ ]:
for metric in metrices:
    count = 0
    for key in bell_results[metric].keys():
        try:
            if  bell_results[metric][key]['self'] <= bell_results[metric][key]['bell']:
                count += 1
        except:
            continue
    print(metric, "wins", count)

In [ ]:
bell_results